In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

# LLM 프롬프트로 Anki 단어장 생성하기

## 문제
###다음 요구사항을 만족하는 LLM 프롬프트를 작성하시오.

## 요구사항
1. 시스템 지시사항

- LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
- 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
- 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
- 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

In [3]:
passage = 'The committee will convene next week to discuss the new policy proposals and reach a consensus.'

# 역할 / 페르소나 / 지시사항
system_message = """
────────────────────────────────────────
[System]  ★지시사항
You are a professional vocabulary-flashcard generator for Anki.
Task: From the English passage provided, extract high-value words or phrases
(academic, technical terms, idioms, proper nouns). Skip very easy words.
For each selected item, output one line in the exact format:

word or phrase;meaning (English or Korean);part of speech;English example sentence

Rules
1. Part of speech: noun, verb, adj, adv 등 소문자 한 단어로 표기
2. Example sentence must be natural English; prefer one from the passage, or write your own.
3. Maximum 40 cards.
4. Output ONLY the cards—no headings, notes, blank lines, or extra commentary.

────────────────────────────────────────
[Example]
Input passage:
<TEXT>
The committee will convene next week to discuss the new policy proposals and
reach a consensus.
</TEXT>

Expected output lines:
convene;to come together for a meeting;verb;The committee will convene next week.
consensus;general agreement;noun;The group reached a consensus after a long discussion.
proposal;a suggestion or plan;noun;He submitted a proposal for the new project.

────────────────────────────────────────
[User  입력]
<TEXT>
The committee will convene next week to discuss the new policy proposals and reach a consensus.
</TEXT>
────────────────────────────────────────

"""

user_message = f"""
다음 텍스트를 anki단어장으로 바꿔주세요.

{passage}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
output = response.choices[0].message.content
print(output)

convene;to come together for a meeting;verb;The committee will convene next week.  
consensus;general agreement;noun;The group reached a consensus after a long discussion.  
proposal;a suggestion or plan;noun;The committee discussed new policy proposals.  


In [5]:
import pandas as pd

rows = []
for line in output.strip().split('\n'):
    parts = line.split(';')
    if len(parts) == 4:
        word, definition, pos, example_sentence = parts
        rows.append({'Word/Phrase': word, 'Definition': definition, 'POS': pos, 'Example_sentence': example_sentence})

df = pd.DataFrame(rows)
df

,Word/Phrase,Definition,POS,Example_sentence
0,convene,to come together for a meeting,verb,The committee will convene next week.
1,consensus,general agreement,noun,The group reached a consensus after a long dis...
2,proposal,a suggestion or plan,noun,The committee discussed new policy proposals.
